In [16]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
import pickle
import obspy
from scipy.signal import find_peaks
import os
import pandas as pd
import math
from scipy.special import rel_entr

import cmcrameri.cm as cmc

from matplotlib import patches


In [8]:
def plot_scatter_figures(proportion, root_times, moments, durations):
    root_times = np.array(root_times)
    moments = np.array(moments)
    durations = np.array(durations)

    plt.scatter(root_times, durations, c = np.log10(moments), cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('Duration (s)')
    plt.xlabel(f'time to release {proportion*100}% of moment (s)')
    plt.colorbar(label = 'log10(moment)')
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/time_for_{proportion*100}_percent_moment_against_duration.png')
    plt.close()

    plt.scatter(root_times, np.log10(moments), c = durations, cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('log10(moment)')
    plt.xlabel(f'time to release {proportion*100}% of moment (s)')
    plt.colorbar(label = 'Duration (s)')
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/time_for_{proportion*100}_percent_moment_against_moment.png')
    plt.close()

    plt.scatter(root_times/durations, np.log10(moments), c = durations, cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('log10(moment)')
    plt.xlabel(f'proportion of duration to release {proportion*100}% of moment')
    plt.colorbar()
    plt.xlim(0, 1)
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/fraction_of_duration_for_{proportion*100}_percent_moment_against_moment.png')
    plt.close()

In [9]:
def myround(x, base=5):
    return base * round(x/base)

In [10]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [11]:
def get_stf(scardec_name, wanted_type = 'fctopt'):
    db = combined[combined['scardec_name']==scardec_name]

    time = []
    momentrate = []

    event = os.listdir(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}')
    starts = [n for n, l in enumerate(event) if l.startswith(wanted_type)]
    with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
        lines = f.read().splitlines()

    lines = lines[2:]
    for line in lines:
        split = line.split(' ')
        split = [s for s in split if s not in ['', ' ', '\n']]
        time.append(float(split[0]))
        momentrate.append(float(split[1]))

    momentrate = np.array(momentrate)
    return momentrate, time, db

In [12]:
# looks for time value of root
def f3(end_time, total_moment, time_opt, momentrate_opt, start, points_before_zero, proportion = 0.1):
    dx = time_opt[1]-time_opt[0]
    end_window = (end_time/dx)+points_before_zero
    end = int(np.floor(end_window))
    short = scipy.integrate.simpson(momentrate_opt[start:end], dx = dx)
    return short-(total_moment*proportion)

In [13]:
def plot_hist_figures(proportion, root_times, durations):
    root_times = np.array(root_times)
    durations = np.array(durations)

    plt.hist(root_times/durations, bins = 100)

    plt.ylabel('Frequency')
    plt.xlabel(f'Proportion of duration to release {proportion*100}% of moment')
    plt.xlim(0, 1)
    plt.show()
    #plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/histogram_fraction_of_duration_for_{proportion*100}_percent_moment.png')
    #plt.close()

In [14]:
def hellinger_explicit(p, q):
    """Hellinger distance between two discrete distributions.
       Same as original version but without list comprehension

    Args:
        p, q: two discrete probability distributions

    Returns:
        Hellinger distance between p and q. This is distributed between 0 and 1.
    """
    p = np.array(p) # these need to be probablility distributions
    q = np.array(q)

    list_of_squares = []
    #for p_i, q_i in zip(p, q):

    # caluclate the square of the difference of ith distr elements
    s = (np.sqrt(p) - np.sqrt(q)) ** 2

    # calculate sum of squares
    sosq = sum(s)

    return np.sqrt(sosq) / math.sqrt(2)

In [94]:
proportions_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
all_durations = []
all_root_times = []
all_moments = []
all_relative_root_times = []

for proportion in proportions_list:
    print(proportion)
    durations = []
    root_times = []
    relative_root_times = []

    diff = []
    moments = []

    for scardec_name in os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec'):
        #print(scardec_name)
        momentrate_opt, time_opt, db = get_stf(scardec_name)

        not_zero = np.where(momentrate_opt > 0)[0]

        dx = time_opt[1]-time_opt[0]

        start = min(not_zero)
        end = max(not_zero)
        points_before_zero = abs(min(time_opt)/dx)

        duration = time_opt[end] - time_opt[start]
        durations.append(duration)

        start_time = time_opt[start]
        end_time = time_opt[end]

        total_moment = scipy.integrate.simpson(momentrate_opt[start:end], dx = time_opt[1]-time_opt[0])
        moments.append(total_moment)
        root, r = scipy.optimize.bisect(f3,
                                        start_time+dx,
                                        end_time,
                                        rtol = 1e-6,
                                        full_output = True,
                                        args = (total_moment,
                                                time_opt,
                                                momentrate_opt,
                                                start,
                                                points_before_zero,
                                                proportion,))
        root_idx = np.floor(root/dx)
        root_time = root_idx*dx
        root_times.append(root_time)
        relative_root_times.append(root_time-start_time)

        if root_time-start_time > duration:
            print('root time greater than duration, proportion:', proportion)
            print(scardec_name)

    root_times = np.array(root_times)
    durations = np.array(durations)
    moments = np.log10(np.array(moments))

    rel_root_times = root_times/durations

    bs = bootstrap(rel_root_times)

    all_n = []
    for bs_run in bs:
        n, bins = np.histogram(bs_run, bins=np.arange(0, 1.01, 0.01))
        all_n.append(n)

    all_n = np.array(all_n)
    all_n_transpose = all_n.T

    plt.stairs(np.mean(all_n_transpose, axis = 1), np.arange(0, 1.01, 0.01))
    plt.fill_between(np.arange(0, 1, 0.01),
                    np.mean(all_n_transpose, axis = 1)+np.std(all_n_transpose, axis = 1),
                    np.mean(all_n_transpose, axis = 1)-np.std(all_n_transpose, axis = 1),
                    step='post', alpha = 0.2, color = 'tab:blue')
    plt.fill_between(np.arange(0, 1, 0.01),
                    np.mean(all_n_transpose, axis = 1)+2*np.std(all_n_transpose, axis = 1),
                    np.mean(all_n_transpose, axis = 1)-2*np.std(all_n_transpose, axis = 1),
                    step='post', alpha = 0.15, color = 'tab:blue')
    plt.fill_between(np.arange(0, 1, 0.01),
                    np.mean(all_n_transpose, axis = 1)+3*np.std(all_n_transpose, axis = 1),
                    np.mean(all_n_transpose, axis = 1)-3*np.std(all_n_transpose, axis = 1),
                    step='post', alpha = 0.1, color = 'tab:blue')#

    rel_root_times

    n, bins = np.histogram(rel_root_times, bins=np.arange(0, 1.01, 0.01))

    plt.stairs(n, bins)

    plt.title(f'Proportion of duration to release {proportion*100:.0f}% of moment')

    plt.ylabel('Frequency')
    plt.xlabel('Proportion of duration to release moment')

    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/bootstrapping_duration_proportions/bootstrapped_histogram_fraction_of_duration_for_{proportion*100:.0f}_percent_moment.png')
    plt.close()

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


In [35]:
def bootstrap(data, n=1000):
    """Bootstrap resampling of data.

    Args:
        data: 1D array of data to be resampled.
        n: number of resamples to take.

    Returns:
        resampled data.
    """

    rng = np.random.default_rng()

    resampled_data = rng.choice(data, (n, len(data)), replace = True)

    return resampled_data